<table style="float:left; border:none">
   <tr style="border:none; background-color: #ffffff">
       <td style="border:none">
           <a href="http://bokeh.org/">     
           <img 
               src="assets/bokeh-transparent.png" 
               style="width:50px"
           >
           </a>    
       </td>
       <td style="border:none">
           <h1>Bokeh Tutorial</h1>
       </td>
   </tr>
</table>

<div style="float:right;"><h2>02. Styling and Theming</h2></div>

In this chapter we will learn how to configure various visual aspects of our plots, and how to find out more about what an be configured. 

First, let's make the standard imports

In [5]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
import numpy as np
import pandas as pd

In [2]:
output_notebook()

Loading BokehJS ...

Before we get started, it's useful to describe how colors and properties are specified in Bokeh. 

## Colors

There are many places where you may need to specify colors. Bokeh can accept colors in a variety of different ways:

* any of the [147 named CSS colors](http://www.w3schools.com/cssref/css_colornames.asp), e.g ``'green'``, ``'indigo'``
* an RGB(A) hex value, e.g., ``'#FF0000'``, ``'#44444444'``
* a 3-tuple of integers *(r,g,b)* between 0 and 255
*  a 4-tuple of *(r,g,b,a)* where *r*, *g*, *b* are integers between 0 and 255 and *a* is a floating point value between 0 and 1



## Properties

Regardless of how a Bokeh plot is created, styling the visual aspects of the plot can always be accomplished by setting attributes on the Bokeh objects that comprise the resulting plot. Visual properties come in three kinds: line, fill, and text properties. For full information with code and examples see the [Styling Visual Properties](http://bokeh.pydata.org/en/latest/docs/user_guide/styling.html) section of the user guide. 

----

### Line Properties

Set the visual appearance of lines. The most common are ``line_color``, ``line_alpha``, ``line_width`` and ``line_dash``.

### Fill Properties

Set the visual appearance of filled areas: ``fill_color`` and ``fill_alpha``.

### Text Properties

Set the visual appearance of lines of text. The most common are ``text_font``, ``text_font_size``, ``text_color``, and ``text_alpha``.

----

Sometimes a prefix is used with property names, e.g. to distinguish between different line properties on the same object, or to give a more meaningful name. For example, to set the line width of the plot outline, you would say ``myplot.outline_line_width = 2``. 

# Plots

Many top-level attributes of plots (outline, border, etc.) can be configured. See the [Plots](http://bokeh.pydata.org/en/latest/docs/user_guide/styling.html#plots) section of the styling guide for full information. 

Here is an example that tweaks the plot outline:

In [3]:
# create a new plot with a title
p = figure(plot_width=400, plot_height=400)
p.outline_line_width = 7
p.outline_line_alpha = 0.3
p.outline_line_color = "navy"

p.circle([1,2,3,4,5], [2,5,8,2,7], size=10)

show(p)

In [7]:
# EXERCISE Create a plot of your own and customize several plot-level properties
# EXERCISE: Try changing the example above to set a `radius` value instead of `size`

# create a new plot with default tools, using figure
p = figure(plot_width=1250, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle([np.random.randint(10,50) for i in range(40)], [np.random.randint(10,50) for i in range(40)], size=[np.random.randint(10,50) for i in range(40)], line_color="pink", fill_color="blue", fill_alpha=np.random.random_sample((40,)))

show(p) # show the results

# Glyphs

It's also possible to style the visual properties of glyphs. When using `bokeh.plotting` this is often done when calling the glyph methods:
```python
p.circle(line_color="red", fill_alpha=0.2, ...)
```
But it is also possible to set these properties directly on glyph objects. Glyph objects are found on `GlyphRenderer` objects, which are returned by the `Plot.add_glyph` and `bokeh.plotting` glyph methods like `circle`, `rect`, etc. Let's look at an example:

In [13]:
p = figure(plot_width=400, plot_height=400)

# keep a reference to the returned GlyphRenderer
r = p.circle([1,2,3,4,5], [2,5,8,2,7])

r.glyph.size = 50
r.glyph.fill_alpha = 0.2
r.glyph.line_color = "firebrick"
r.glyph.line_dash = [5, 100]
r.glyph.line_width = 20

show(p)

### Selection and non-selection visuals

You can also control how glyphs look when there are selections involved. The set of "selected" points is displayed according to the optional `.selection_glyph` property of a `GlyphRenderer`:
```python
r.selection_glyph = Circle(fill_alpha=1, fill_color="firebrick", line_color=None) 
```
When there is a non-empty selection, the set of "unselected: points is displayed according to the optional  `.nonselection_glyph` property of a `GlyphRenderer`:
```python
r.nonselection_glyph = Circle(fill_alpha=0.2, fill_color="grey", line_color=None) 
```

When using the `bokeh.plotting` interface, it is easier to pass these visual properties to the glyph methods as shown below. The glyph method will create the selection or nonselection glyphs and attach them to the renderer for you. 

In [14]:
p = figure(plot_width=400, plot_height=400, tools="tap", title="Select a circle")
renderer = p.circle([1, 2, 3, 4, 5], [2, 5, 8, 2, 7], size=50,

                    # set visual properties for selected glyphs
                    selection_color="firebrick",

                    # set visual properties for non-selected glyphs
                    nonselection_fill_alpha=0.2,
                    nonselection_fill_color="grey",
                    nonselection_line_color="firebrick",
                    nonselection_line_alpha=1.0)

show(p)

It is also possible to specify the visual appearance of glyphs when they are "inspected", e.g. by a hover tool. This is accomplished by setting an optional `hover_glyph` on the glyph renderer:
```python
r.hover_glyph = Circle(fill_alpha=1, fill_color="firebrick", line_color=None) 
```
Or if using `bokeh.plotting` glyph methods, by passing `hover_fill_alpha`, etc. to the glyph method. Lets look at an example that works together with a `HoverTool` configured for "hline" hit-testing.

In [15]:
from bokeh.models.tools import HoverTool
from bokeh.sampledata.glucose import data

subset = data.loc['2010-10-06']

x, y = subset.index.to_series(), subset['glucose']

# Basic plot setup
p = figure(width=600, height=300, x_axis_type="datetime", title='Hover over points')

p.line(x, y, line_dash="4 4", line_width=1, color='gray')

cr = p.circle(x, y, size=20,
              fill_color="grey", hover_fill_color="firebrick",
              fill_alpha=0.05, hover_alpha=0.3,
              line_color=None, hover_line_color="white")

p.add_tools(HoverTool(tooltips=None, renderers=[cr], mode='hline'))

show(p)

In [27]:
# EXERCISE: experiment with standard, selected, hover glyph visual properties
from bokeh.models.tools import HoverTool
from bokeh.sampledata.glucose import data

subset = data.loc['2010-10-06']

x, y = subset.index.to_series(), subset['glucose']

# Basic plot setup
p = figure(width=600, height=300, x_axis_type="datetime", title='Hover over points')

p.line(x, y, line_dash="15 20", line_width=7, color='gray')

cr = p.circle(x, y, size=20,
              fill_color="grey", hover_fill_color="firebrick",
              fill_alpha=0.05, hover_alpha=0.3,
              line_color=None, hover_line_color="navy")

p.add_tools(HoverTool(tooltips=None, renderers=[cr], mode='hline'))

show(p)

# Axes

Next we will take a look at [stlying of Axes](http://bokeh.pydata.org/en/latest/docs/user_guide/styling.html#axes).

To style axes, you first must get ahold of `Axis` objects. The simplest way is to use some convenience methods on `Plot`: [axis](http://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.Figure.axis), [xaxis](http://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.Figure.xaxis), and [yaxis](http://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.Figure.yaxis). These methods return lists of axis objects:

```
>>> p.xaxis
[<bokeh.models.axes.LinearAxis at 0x106fa2390>]
```
However, you can set properties on all the elements of the list as if it was a single object:
```
p.xaxis.axis_label = "Temperature"
p.axis.major_label_text_color = "orange"
```
These are referred to as "splattable" lists, and tab completion works on them as well. 


In [9]:
# EXERCISE Try out tab completion. Type p.xaxis.<press tab key> to see a list of attributes that can be set.
#la complétion ne marche pas !? Oo



## Axis properties

Axes objects have many configurable properties that afford control over most visual aspects of an axis. These can be grouped by function according to prefix:

* **axis**  [line properties](http://bokeh.pydata.org/en/latest/docs/user_guide/styling.html#line-properties) e.g `axis_line_width`

* **axis_label** [text properties](http://bokeh.pydata.org/en/latest/docs/user_guide/styling.html#text-properties)  e.g. `axis_label_text_color`, as well as ``axis_label_standoff``

* **major_label** [text properties](http://bokeh.pydata.org/en/latest/docs/user_guide/styling.html#text-properties) e.g. `major_label_text_font_size`, as well as ``major_label_orientation``

* **major_tick** [line_properties](http://bokeh.pydata.org/en/latest/docs/user_guide/styling.html#line-properties) e.g. `major_tick_line_dash`, as well as  ``major_tick_in`` and ``major_tick_out``

* **minor_tick** [line properties](http://bokeh.pydata.org/en/latest/docs/user_guide/styling.html#line-properties) e.g. `minor_tick_line_width`, as well as ``minor_tick_in`` and ``minor_tick_out``

As a simple first case, let's change the orientation of the major tick labels on both axes of a plot:

In [28]:
from math import pi

p = figure(plot_width=400, plot_height=400)
p.x([1,2,3,4,5], [2,5,8,2,7], size=10, line_width=2)

p.xaxis.major_label_orientation = pi/4
p.yaxis.major_label_orientation = "vertical"

show(p)

The next example shows customizations on several of the different Axis properties at once:

In [29]:
p = figure(plot_width=400, plot_height=400)
p.asterisk([1,2,3,4,5], [2,5,8,2,7], size=12, color="olive")

# change just some things about the x-axes
p.xaxis.axis_label = "Temp"
p.xaxis.axis_line_width = 3
p.xaxis.axis_line_color = "red"

# change just some things about the y-axes
p.yaxis.axis_label = "Pressure"
p.yaxis.major_label_text_color = "orange"
p.yaxis.major_label_orientation = "vertical"

# change things on all axes
p.axis.minor_tick_in = -3
p.axis.minor_tick_out = 6

show(p)

In [35]:
# EXERCISE Create a plot of your own and customize several axis properties
# EXERCISE Create a plot of your own and customize several plot-level properties
# EXERCISE: Try changing the example above to set a `radius` value instead of `size`

# create a new plot with default tools, using figure
p = figure(plot_width=1250, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.diamond([np.random.randint(10,50) for i in range(40)], [np.random.randint(10,50) for i in range(40)], size=[np.random.randint(10,50) for i in range(40)], line_color="pink", fill_color="blue", fill_alpha=np.random.random_sample((40,)))
# change just some things about the x-axes
p.xaxis.axis_label = "r4ndom"
p.xaxis.axis_line_width = 10
p.xaxis.axis_line_color = "teal"

# change just some things about the y-axes
p.yaxis.axis_label = "r4ndom"
p.yaxis.major_label_text_color = "navy"
p.yaxis.major_label_orientation = "horizontal"

# change things on all axes
p.axis.minor_tick_in = 3
p.axis.minor_tick_out = 3
show(p) # show the results

## Configuring tick labels

All Bokeh axes have a `formatter` property, whose value is a `TickFormatter` object that Bokeh uses to format the ticks displayed by that axis. Bokeh will configure default tick formatters for numeric, datetime, or categotical axes. But often we would like to customize the appearance of tick labels. This can be accomplished by changing properties on the default formatter that Bokeh chooses, or by replacing the formatter with a new type entirely. 

Let's first look at changing the properties of a default formatter. The default datetime formatter is configure to show *month/day* when the axis is on the scale of days. If would like the also always show the year, we can change the `days` property to a format that includes the year, as done below. 

In [39]:
from math import pi
from bokeh.sampledata.glucose import data

week = data.loc['2010-10-01':'2010-10-08']

p = figure(x_axis_type="datetime", title="Glocose Range", plot_height=350, plot_width=800)
p.xaxis[0].formatter.days = '%Y/%m/%d'
p.xaxis.major_label_orientation = pi/3

p.line(week.index, week.glucose)

show(p)

See the reference guide entry for [DatetimeTickFormatter](http://bokeh.pydata.org/en/latest/docs/reference/models/formatters.html#bokeh.models.formatters.DatetimeTickFormatter) to see other properties that can be updated. 

*Note: It's worth noting a little quirk here, because of the way "splattable" lists work, we do have to be explicit about which formatter object we want to update, i.e. `p.xaxis[0].formatter`*

In addition to the tick formatters that Bokeh will use by default, there are others such as the [`NumeralTickFormatter`](http://bokeh.pydata.org/en/latest/docs/user_guide/styling.html#numeraltickformatter) that we can configure explicitly. The example below shows how to set a formatter on each axis. 

In [40]:
from bokeh.models import NumeralTickFormatter

p = figure(plot_height=300, plot_width=800)
p.circle([1,2,3,4,5], [2,5,8,2,7], size=10)

p.xaxis.formatter = NumeralTickFormatter(format="0.0%")
p.yaxis.formatter = NumeralTickFormatter(format="$0.00")

show(p)

Try experimenting with the format argument and re-execute the cell above.

There are many other possibilities for controlling tick formatting, including the possibility of supplying a JavaScript snippet to perform arbitrary formatting in the browser. See the [Tick Label Formats](http://bokeh.pydata.org/en/latest/docs/user_guide/styling.html#tick-label-formats) for more details.  


It is also possible to customize *where* ticks will be drawn. See the [Tick Locations](http://bokeh.pydata.org/en/latest/docs/user_guide/styling.html#tick-locations) section of the User's Guide for more information.

# Grids

It is also possible to control the [styling of Grids](http://bokeh.pydata.org/en/latest/docs/user_guide/styling.html#grids)

Grids properties in Bokeh have two possible prefixes:

* **grid** properties (which are [line properties](http://bokeh.pydata.org/en/latest/docs/user_guide/styling.html#line-properties)) control the "grid lines"
* **band** properties (which are [fill properties](http://bokeh.pydata.org/en/latest/docs/user_guide/styling.html#fill-properties)) control shaded bands between grid lines

In this first example we turn off the vertical grid lines (by setting the line color to None) and set the horizontal grid to be light and dashed.

In [41]:
p = figure(plot_width=400, plot_height=400)
p.circle([1,2,3,4,5], [2,5,8,2,7], size=10)

# change just some things about the x-grid
p.xgrid.grid_line_color = None

# change just some things about the y-grid
p.ygrid.grid_line_alpha = 0.5
p.ygrid.grid_line_dash = [6, 4]

show(p)

The next example shows how the "band" properties of a plot can be specified

In [42]:
p = figure(plot_width=400, plot_height=400)
p.circle([1,2,3,4,5], [2,5,8,2,7], size=10)

# change just some things about the x-grid
p.xgrid.grid_line_color = None

# change just some things about the y-grid
p.ygrid.band_fill_alpha = 0.1
p.ygrid.band_fill_color = "navy"

show(p)

In [47]:
# EXERCISE Create a plot of your own and customize several grid properties
# EXERCISE Create a plot of your own and customize several plot-level properties
# EXERCISE: Try changing the example above to set a `radius` value instead of `size`

# create a new plot with default tools, using figure
p = figure(plot_width=1250, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.diamond_cross([np.random.randint(10,50) for i in range(40)], [np.random.randint(10,50) for i in range(40)], size=[np.random.randint(10,50) for i in range(40)], line_color="pink", fill_color="blue", fill_alpha=np.random.random_sample((40,)))
# change just some things about the x-grid
p.xgrid.grid_line_color = None

# change just some things about the y-grid
p.ygrid.band_fill_alpha = 0.5
p.ygrid.band_fill_color = "teal"

p.xgrid.band_hatch_pattern = "/"
p.xgrid.band_hatch_alpha = 0.6
p.xgrid.band_hatch_color = "lightgrey"
p.xgrid.band_hatch_weight = 0.5
p.xgrid.band_hatch_scale = 10


show(p) # show the results